# Description
Example of a first passage time computation for a twisted state.

In [ ]:
using Plots
using Printf
using Random
using LaTeXStrings
using DSP
using LinearAlgebra
using ForwardDiff
using Optimization
using OptimizationOptimJL
using Plots.Measures

In [ ]:
default(xtickfontsize=14,  ytickfontsize=14, 
    guidefontsize=14, 
    legendfontsize=12, lw=2,ms=4)

In [ ]:
push!(LOAD_PATH, "../GeneralizedKuramoto");
push!(LOAD_PATH, "../QProcesses");
push!(LOAD_PATH, "../GraphMatrices");
push!(LOAD_PATH, "../TwistedStates");
push!(LOAD_PATH, "../ClassicalKuramoto");

In [ ]:
using GeneralizedKuramoto
using QProcesses
using GraphMatrices
using TwistedStates
using ClassicalKuramoto

# Parameter choices

In [ ]:
n = 10;         # number of sites
k = 1;          # interaction range
q = 1;          # initial twisted state
Δt = 1e-2;      # time step
β = 40.0;       # inverse temperature
α = 0;          # attractive interactions
X = 1.0;        # spatial domain [0,1)
tmax = 10^2;    # time of integration
Δt_save = Δt;   # frequency of saving


# Prepare data

In [ ]:
u0 = mod.(TwistedStates.construct_q_twisted(n, q), 1);

# Set solver for integration

In [ ]:
maxΔt = Int(tmax / Δt);
nsave = Int(Δt_save / Δt);

# prepare interaction matrix
K = 1 / (2 * k + 1) * GraphMatrices.discrete_knn_spmatrix(n, k);
# K = GraphMatrices.discrete_knn_spmatrix(n, k)

σ = sqrt(2/β);
# prepare sampler
Q_sampler = let σ = σ, n = n
    () -> QProcesses.IsoGaussian(σ, n);
end
# prepare nonlinearity
f = (t, u) -> 0.0
# coupling
S = let α = α
    (u, v) -> (-1)^(α) * sinpi(2 * (v - u));
end


Construct functions for detecting transitions out of the state (well).  At each time step:
* Given the current values of $\{u_i(t)\}$ stored in vector `u`, minimize the energy by BFGS methods, with this `u` as a the initial guess.
* When the minimized energy changes from the current value, stored in `Einit`, we conclude that the system has switched basins of attraction on the energy landscape; we have shifted states.

In [ ]:
# energy function
E = u -> ClassicalKuramoto.energy(u, K, α);

# optimization function
opt_func = OptimizationFunction((v, p) -> E([0; v]), Optimization.AutoForwardDiff())
opt_prob = OptimizationProblem(opt_func, u0[2:end])
soln = solve(opt_prob, BFGS())
Einit = soln.objective

function in_well(u; tol=1e-6)
    opt_prob.u0 .= unwrap(u .- u[1], range=1)[2:end]
    soln = solve(opt_prob, BFGS())
    return Bool(abs(soln.objective - Einit) < tol)
end


# Integrate the trajectory

In [ ]:
Random.seed!(123);
t_trajectory, u_trajectory = GeneralizedKuramoto.EulerMaruyamaFPT(u0, in_well, f, K, S, Q_sampler, Δt, maxΔt, nsave=nsave, verbose=false);
println("First exit time");
@show t_trajectory[end];


# Visualize trajectory

In [ ]:
anim = @animate for i=1:length(t_trajectory)
    scatter(1:n, mod.(u_trajectory[i],1),label="")
    ylims!(-0.1, 1.1);
    title!(@sprintf("t = %g", t_trajectory[i]));
end
gif(anim, fps = 15)